In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
import time
from sqlalchemy import text, inspect, MetaData, Table
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


In [2]:
Session = sessionmaker(bind=engine)
session = Session()
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()
session.close()

accounts = getTables(engine)

for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    
    Session = sessionmaker(bind=engine)
    session = Session()

    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)
    allFiles.to_sql('transactions_all', con=session.connection(), if_exists='append', index=False, chunksize=10)
    session.connection().commit()
    session.close()


In [3]:
# inspector = inspect(engine)
# tables = inspector.get_table_names()
# for table in tables:
#     print(table)

# metadata = MetaData()
# metadata.reflect(bind=engine)

# def listCols(tablename):
#     table = metadata.tables[tablename]
#     print(tablename)
#     for column in table.columns:
#         print(f"   {column}")

# listCols('accounts')
# listCols('loads')


pd.read_sql(sql=text("select * from transactions_all"), con=engine.connect())   

    

In [5]:
session.connection().execute(text("DELETE FROM loads"))
session.connection().execute(text("DELETE FROM transactions_all"))
session.connection().commit()